In [ ]:
#Checking if GPU is running or not

!nvidia-smi

Mon Nov 27 15:33:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

## Helsinki-NLP/opus-mt-en-hi model

source: https://huggingface.co/Helsinki-NLP/opus-mt-en-hi



# The Dataset

Source: https://huggingface.co/datasets/cfilt/iitb-english-hindi

In [ ]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [ ]:
raw_datasets['train'][1]

{'translation': {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}}

#Preprocessing the data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [12110, 2, 90, 23, 19, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[12110, 2, 90, 23, 19, 8800, 61, 0], [239, 23, 414, 8800, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["एक्सेर्साइसर पहुंचनीयता अन्वेषक"]))

{'input_ids': [[26618, 16155, 346, 33383, 0]], 'attention_mask': [[1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset, DatasetDict, Dataset, concatenate_datasets

# Assuming raw_datasets is already loaded
# If not, you can load it using load_dataset('cfilt/iitb-english-hindi')
# raw_datasets = load_dataset('cfilt/iitb-english-hindi')

# Split the train dataset into train and validation with a ratio of 80:20
train_dataset, val_dataset = train_test_split(
    raw_datasets['train']['translation'][-30000:],
    test_size=0.2,
    random_state=42  # Set a random seed for reproducibility
)

test_dataset = concatenate_datasets([raw_datasets['test'], raw_datasets['validation']])
print(test_dataset)
print({'translation': val_dataset})

# Create new DatasetDict with the split datasets
new_raw_datasets = DatasetDict({
    'train': Dataset.from_dict({'translation': train_dataset}),
    'validation': Dataset.from_dict({'translation': val_dataset}),
    'test': test_dataset,
})

# Verify the changes
print(new_raw_datasets)


Dataset({
    features: ['translation'],
    num_rows: 3027
})
{'translation': [{'en': 'It was created as a conservation area in 1999 and was designated as a biosphere reserve by UNESCO in 2009.', 'hi': 'यह 1999 में एक संरक्षण क्षेत्र के रूप में बनाया गया था और 2009 में यूनेस्को द्वारा एक बायोस्फीयर रिजर्व के रूप में नामित किया गया था।'}, {'en': 'VENU in South India', 'hi': 'दक्षिण भारत में वेणु'}, {'en': 'His teachings make us large-hearted.', 'hi': 'उनकी शिक्षाएँ हमें बड़ा दिलवाला बनाती हैं।'}, {'en': 'We will have to evolve our own patterns of development and physical growth, our own methods and materials of construction and our own expression of foregoing.', 'hi': 'हमें स्वयं अपने विकास और प्राकृतिक वृद्धि\u200d के प्रतिमान, निर्माण के अपने तरीके और सामग्री तथा पूर्ववर्ती अभिव्यक्ति विकसित करनी पड़ेंगी ।'}, {'en': 'He specialized in portraits, which broke many a rule of the court, as we can see from the photographs he took of the residents of the Zenana, thus giving us a glimpse of

In [ ]:
print(new_raw_datasets['test']['translation'][:10])
print(new_raw_datasets['train']['translation'][:10])

[{'en': 'A black box in your car?', 'hi': 'आपकी कार में ब्लैक बॉक्स?'}, {'en': "As America's road planners struggle to find the cash to mend a crumbling highway system, many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car.", 'hi': 'जबकि अमेरिका के सड़क योजनाकार, ध्वस्त होते हुए हाईवे सिस्टम को सुधारने के लिए धन की कमी से जूझ रहे हैं, वहीं बहुत-से लोग इसका समाधान छोटे से ब्लैक बॉक्स में देख रहे हैं, जो आपकी कार के डैशबोर्ड पर सफ़ाई से फिट हो जाता है।'}, {'en': "The devices, which track every mile a motorist drives and transmit that information to bureaucrats, are at the center of a controversial attempt in Washington and state planning offices to overhaul the outdated system for funding America's major roads.", 'hi': 'यह डिवाइस, जो मोटर-चालक द्वारा वाहन चलाए गए प्रत्येक मील को ट्रैक करती है तथा उस सूचना को अधिकारियों को संचारित करती है, आजकल अमेरिका की प्रमुख सड़कों का वित्त-पोषण करने के लिए पुराने हो चुके सिस्टम का जीर्णोद्धार करने के

In [ ]:
import pandas as pd

# Convert datasets to pandas dataframes
train_df = pd.DataFrame(new_raw_datasets['train']['translation'])
val_df = pd.DataFrame(new_raw_datasets['validation']['translation'])
test_df = pd.DataFrame(new_raw_datasets['test']['translation'])

# Save dataframes to CSV files
train_df.to_csv('train_dataset.csv', index=False)
val_df.to_csv('validation_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)


In [ ]:
preprocess_function(new_raw_datasets["train"][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[239, 43246, 9024, 2198, 1758, 12187, 3931, 4568, 1347, 137, 1211, 5192, 8, 20055, 16, 2, 41577, 16, 2, 14500, 18008, 10, 20365, 13186, 3, 0], [20539, 865, 865, 9043, 18052, 10081, 1053, 345, 3070, 2100, 596, 13834, 389, 27730, 7621, 2906, 524, 901, 11503, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[89, 13922, 1185, 45, 9703, 1908, 3904, 11, 1758, 1108, 26139, 3547, 28, 2, 2557, 10553, 45, 314, 438, 6, 6357, 9059, 21004, 2, 88, 2698, 25412, 11195, 260, 966, 2, 67, 24887, 1185, 45, 1321, 18535, 1231, 4985, 3695, 1185, 45, 1056, 716, 32550, 18136, 86, 18, 2001, 78, 115, 5, 14, 0], [8217, 11781, 36467, 1321, 20507, 5066, 0]]}

In [ ]:
tokenized_datasets = new_raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3027 [00:00<?, ? examples/s]

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)


In [ ]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [ ]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

1500/1500 [==============================] - 493s 301ms/step - loss: 3.0623 - val_loss: 2.5569


In [ ]:
model.save_pretrained("tf_model/")

# Model Testing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
import tensorflow as tf
def get_attention_weights(output):
    attention_weights = output['encoder_attentions'][-1][0]  # Taking attention weights from the last layer
    if isinstance(attention_weights, tf.Tensor):
        attention_weights = attention_weights.numpy()
    attention_weights = tf.math.reduce_sum(attention_weights, axis=[1, 2])  # Summing across heads and tokens
    attention_weights /= tf.reduce_sum(attention_weights)  # Normalize attention weights
    return attention_weights

def eng_to_hindi(input_text):
    tokenized = tokenizer([input_text], return_tensors='tf')
    out = model.generate(**tokenized, max_length=128, output_attentions=True)
    with tokenizer.as_target_tokenizer():
        return tokenizer.decode(out[0], skip_special_tokens=True)
    # print(get_attention_weights(out))
print(eng_to_hindi("Tom how are you?"))
print(len(new_raw_datasets["test"]["translation"]))


Tom आप कैसे हैं?
3027


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
import pandas as pd
from tqdm import tqdm

# Subset of examples
examples_subset = new_raw_datasets["test"]["translation"]

# Generate Hindi translations
translations = []
for example in examples_subset:
    eng_text = example['en']
    hindi_translation = eng_to_hindi(eng_text)
    translations.append({"English": eng_text, "Hindi": hindi_translation, "Original":example['hi']})

# Create a DataFrame
df = pd.DataFrame(translations)

# Save to CSV
csv_file_path = "translations.csv"
df.to_csv(csv_file_path, index=False, encoding='utf-8')

print(f"Translations saved to {csv_file_path}")


Translations saved to translations.csv
